In [69]:
import pandas as pd
import json
import numpy as np
from google.cloud import bigquery

In [70]:
#conversion du fichier json en json line delimiter pour utiliser pd.read_json avec une chunk size. (Nombre de ligne processée en une fois)
#!cat ../raw_data/export_23-5_minified.json | jq -c '.[]' > ../raw_data/export_converted_2.json

In [71]:
#Localisation du projet name et table_id sur gcp
project="marbotic"
table_id = "marbotic_dataset.user_id"

In [72]:
#intégration des credentials 
from google.oauth2 import service_account

key_path = "/Users/antonin/code/AntoninAnq/gcp/marbotic-7d02fac30bd8.json"

credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

In [87]:
CHUNKSIZE = 5000

file_path = '../raw_data/users_info_202206071551.csv'
df = pd.read_csv(file_path,chunksize=CHUNKSIZE,sep=";")
client = bigquery.Client(project,credentials)

for i,c in enumerate(df):
    ep=pd.DataFrame()
    c.reset_index(inplace=True)
    ep=c.copy()
    ep.drop(['Country','Language','News letter','Type','Games','Products','Pieces','index'],axis=1,inplace=True)
    ep.rename(columns={'User id':'User_id'},inplace=True)

    cat = ep['Purchases'].copy().map(lambda x: ", ".join(x)
                                        if isinstance(x, list) else x)
    names = list(
        set([
            x.strip().strip("'").strip("['")
            for x in ', '.join(', '.join(', '.join(
                list(set([str(x) for x in cat]))).split('\n')).split(
                    "' '")).split(',')
        ]))

    names_transf = [
        'Purchases' + '_' + x.replace(' ', '_').replace(']', '').replace('"', '') for x in names
        if x != 'nan' and x != ''
    ]
    names = [x for x in names if x != 'nan' and x != '']
    for ind, name in enumerate(names_transf):
        ep[f'{name}'] = ep['Purchases'].map(
            lambda x: 1 if isinstance(x, str) and f'{names[ind]}' in x else
            (1 if isinstance(x, list) and
             len([n for n in x if f'{names[ind]}' in n]) > 0
             else 0))
        
    ep.drop(['Purchases'],axis=1,inplace=True)
    ep.to_csv('../raw_data/user_id.csv',index=False)
        
    job = client.load_table_from_dataframe(ep, table_id)  # Make an API request. ,job_config=job_config
    job.result()  # Wait for the job to complete.
    table = client.get_table(table_id)  # Make an API request.
    print(
        "Loaded {} rows and {} columns to {}".format(
            table.num_rows, len(table.schema), table_id
        )
    )

Loaded 5000 rows and 13 columns to marbotic_dataset.user_id
Loaded 10000 rows and 13 columns to marbotic_dataset.user_id
Loaded 13679 rows and 13 columns to marbotic_dataset.user_id


In [79]:
data = pd.read_csv('../raw_data/user_id.csv')
data

,User_id,Attribution,"Purchases_""tier_upgrade_0_to_1""","Purchases_""PLY""]","Purchases_""PLM""]","Purchases_""tier_upgrade_0_to_1""]","Purchases_""MPGR""]","Purchases_""PY3M""]","Purchases_""PM3M""","Purchases_""PM3M""]"
0,59707,PLU,0,0,0,0,0,0,0,0
1,61748,ELU,0,0,0,0,0,0,0,0
2,54834,PLU,0,0,0,0,0,0,0,0
3,69202,PLU,0,0,0,0,0,0,0,0
4,67754,PLU,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
995,55043,ELU,0,0,0,0,0,0,0,0
996,14491,ELU,0,0,0,0,0,0,0,0
997,62119,ELU,0,0,0,0,0,0,0,0
998,54818,ELU,0,0,0,0,0,0,0,0
